In [1]:
import requests
import pandas as pd
import numpy as np
import json
import sqlalchemy
from sqlalchemy import create_engine
from flask import Flask, request, render_template, jsonify
import pymysql
pymysql.install_as_MySQLdb()
from config import remote_db_endpoint, remote_db_port, remote_db_name, remote_db_user, remote_db_pwd
from config import x_rapidapi_key, x_rapidapi_host, spoonacular_API

import pprint
import urllib.request

In [34]:
recipe_ids_list = [1554861,1560677,1571559]

In [35]:
capture_list = []
for recipe_id in recipe_ids_list:
    url2 = f"https://api.spoonacular.com/recipes/{recipe_id}/information?apiKey={spoonacular_API}"    
    response = requests.get(url2)
    response_json = response.json()
    capture_list.append(response_json)

In [36]:
capture_list

[{'vegetarian': False,
  'vegan': False,
  'glutenFree': True,
  'dairyFree': False,
  'veryHealthy': False,
  'cheap': False,
  'veryPopular': False,
  'sustainable': False,
  'weightWatcherSmartPoints': 12,
  'gaps': 'no',
  'lowFodmap': False,
  'preparationMinutes': 60,
  'cookingMinutes': 0,
  'aggregateLikes': 0,
  'spoonacularScore': 5.0,
  'healthScore': 0.0,
  'creditsText': 'Beyond Frosting',
  'sourceName': 'Beyond Frosting',
  'pricePerServing': 27.09,
  'extendedIngredients': [{'id': 19081,
    'aisle': 'Sweet Snacks',
    'image': 'milk-chocolate.jpg',
    'consistency': 'solid',
    'name': 'recipe',
    'nameClean': 'chocolate',
    'original': '1 Recipe for Vanilla or Chocolate cupcakes',
    'originalString': '1 Recipe for Vanilla or Chocolate cupcakes',
    'originalName': 'Recipe for Vanilla or Chocolate cupcakes',
    'amount': 1.0,
    'unit': '',
    'meta': ['for vanilla or chocolate cupcakes'],
    'metaInformation': ['for vanilla or chocolate cupcakes'],
    '

In [37]:
# parsed = json.loads(capture_list[0])
print(json.dumps(capture_list, indent=4, sort_keys=True))

[
    {
        "aggregateLikes": 0,
        "analyzedInstructions": [
            {
                "name": "",
                "steps": [
                    {
                        "equipment": [],
                        "ingredients": [
                            {
                                "id": 18139,
                                "image": "plain-cupcake.jpg",
                                "localizedName": "cupcakes",
                                "name": "cupcakes"
                            }
                        ],
                        "number": 1,
                        "step": "Back a batch of cupcake and cool completely."
                    },
                    {
                        "equipment": [],
                        "ingredients": [
                            {
                                "id": 10119230,
                                "image": "vanilla-frosting.png",
                                "localizedName": "vanilla frosti

In [38]:
recipe_ingredients = []
recipe_steps = []

# ingredients stuff
for result in capture_list:                         ################ NOT SURE BUT PROBABLY
    try:
        recipe_id = result['id']
        recipe_title = result['title']        
        analyzedInstructions = result['analyzedInstructions']

    except Exception as e:
        print('--- error with something ---')
        print(e)
        continue 

    instruction_steps = analyzedInstructions[0]['steps']        # Brooke addition

    counter = 0                                                 # Brooke addition

    # INSTRUCTIONS ##############################
    for item in instruction_steps:                              # Brooke addition
        counter = counter + 1                                   # Brooke addition
        step = item['step']                                     # Brooke addition
        numbered_step = f'{counter}. {step}'                    # Brooke addition
        recipe_steps.append(numbered_step)                      # Brooke addition

    # INGREDIENTS ###############################
    for instruction in analyzedInstructions:

        steps = instruction['steps']

        for step in steps:

            ingredients = step['ingredients']

            for ingredient in ingredients:

                ingredient_name = ingredient['name']

                recipe_ingredient = {
                    'recipe_id': recipe_id,
                    'recipe_title': recipe_title,
                    'ingredient_name': ingredient_name
                }

                recipe_ingredients.append(recipe_ingredient)

ingredients_df = pd.DataFrame(recipe_ingredients)

# dedupe ingredients df
# ingredients_df.drop_duplicates()
ingredients_df.drop_duplicates(subset=['ingredient_name'], inplace=True)

ingredients_df

######################## KEEP FOR POSSIBLE USE WITH FUNCTION
# return ingredients_df

,recipe_id,recipe_title,ingredient_name
0,1554861,Christmas Cupcakes,cupcakes
1,1554861,Christmas Cupcakes,vanilla frosting
2,1554861,Christmas Cupcakes,butter
3,1554861,Christmas Cupcakes,vanilla extract
4,1554861,Christmas Cupcakes,powdered sugar
7,1554861,Christmas Cupcakes,sugar
8,1554861,Christmas Cupcakes,heavy cream
10,1554861,Christmas Cupcakes,frosting
11,1560677,Caramel Apple Slab Pie,brown sugar
12,1560677,Caramel Apple Slab Pie,all purpose flour


In [ ]:
# download raw json object
url = "http://127.0.0.1:5000/ingredientsWithPrices"
data = urllib.request.urlopen(url).read().decode()

# parse json object
obj = json.loads(data)

print(json.dumps(obj, indent=4, sort_keys=True))

In [39]:
cloud_engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")

cloud_conn = cloud_engine.connect()

#%% Querying the database
query = '''
        SELECT DISTINCT
            ingredient,
            price,
            title,
            size
        FROM
            products_subset
        '''

products_subset = pd.read_sql(query, cloud_conn)

products_subset




,ingredient,price,title,size
0,1% fat buttermilk,2.0,Maola Cultured 1% Low Fat Buttermilk,1 QUART
1,1% fat cottage cheese,3.0,Hood Cottage Cheese with Peaches Small Curd Lo...,16 OZ TUB
2,1% fat cottage cheese,1.0,Friendship Dairies Cottage Cheese Small Curd L...,5 OZ TUB
3,1% fat cottage cheese,1.0,Friendship Dairies Cottage Cheese Pineapple Sm...,5 OZ TUB
4,1% fat cottage cheese,1.0,Hood Cottage Cheese with Peaches Low Fat 1% Mi...,5.3 OZ TUB
...,...,...,...,...
91835,zucchini,4.0,Birds Eye Veggie Spirals Zucchini Frozen,10 OZ BOX
91836,zucchini,4.0,Birds Eye Veggie Spirals Zucchini with Marinara,10 OZ BOX
91837,zucchini,3.0,PicSweet Farms Vegetables for Grilling Yellow ...,12 OZ PKG
91838,zucchini,5.0,Birds Eye Zucchini Fries Shoestring,12 OZ BAG


In [7]:
len(products_subset)

91840

In [40]:
# Renamed to GROCERY DF for clarity
# Cut down to a single return for each ingredient
grocery_df = products_subset
grocery_df.drop_duplicates(subset='ingredient', keep='first', inplace=True)
grocery_df = grocery_df.rename(columns={"title": "ingredient_title"})
print(len(grocery_df))
grocery_df.head()

7229


,ingredient,price,ingredient_title,size
0,1% fat buttermilk,2.0,Maola Cultured 1% Low Fat Buttermilk,1 QUART
1,1% fat cottage cheese,3.0,Hood Cottage Cheese with Peaches Small Curd Lo...,16 OZ TUB
11,1% low-fat chocolate milk,4.0,Ahold Icon Giant 1% Low Fat Chocolate Milk,1 GALLON
18,1% low-fat milk,4.0,Ahold Icon Giant 1% Low Fat Milk,1 GALLON
43,10-inch baked pie shells,3.0,Keebler Ready Crust Pie Crust Graham Cracker 1...,9 OZ PAN


In [41]:
recipe_ingredients_df = ingredients_df
recipe_ingredients_df = recipe_ingredients_df.rename(columns={"ingredient_name": "ingredient"})
recipe_ingredients_df.head()

,recipe_id,recipe_title,ingredient
0,1554861,Christmas Cupcakes,cupcakes
1,1554861,Christmas Cupcakes,vanilla frosting
2,1554861,Christmas Cupcakes,butter
3,1554861,Christmas Cupcakes,vanilla extract
4,1554861,Christmas Cupcakes,powdered sugar


In [42]:
new = pd.merge(recipe_ingredients_df, grocery_df, how="inner", on=["ingredient", "ingredient"])
                                                                    
new.head()                                                               

,recipe_id,recipe_title,ingredient,price,ingredient_title,size
0,1554861,Christmas Cupcakes,cupcakes,3.0,Just Desserts Cupcake Red Velvet,4.4 OZ PKG
1,1554861,Christmas Cupcakes,vanilla frosting,2.0,Pillsbury Funfetti Radiant Red Vanilla Frosting,15.6 OZ PKG
2,1554861,Christmas Cupcakes,butter,5.0,Kerrygold Pure Irish Butter Grass-fed Naturall...,8 OZ TUB
3,1554861,Christmas Cupcakes,vanilla extract,8.0,Ahold Icon Giant Pure Vanilla Extract,2 OZ BTL
4,1554861,Christmas Cupcakes,powdered sugar,4.0,Woodstock Powdered Sugar Organic,16 OZ PKG


In [22]:
grocery_df.loc[grocery_df['ingredient'] == "butter"].head(1)    

,ingredient,price,ingredient_title,size
9545,butter,5.0,Kerrygold Pure Irish Butter Grass-fed Naturall...,8 OZ TUB


In [18]:
column_names = ["recipe_id", "recipe_title", "ingredient", "ingredient_title", "price", "size"]
df = pd.DataFrame(columns = column_names)
df

,recipe_id,recipe_title,ingredient,ingredient_title,price,size


In [ ]:
recipe_ingredients_df.merge(grocery_df, how='left', on='ingredient')
recipe_ingredients_df

In [ ]:
empty_list = []
for recipe_ingredient in recipe_ingredients_list:
    matches_df = grocery_df.loc[grocery_df['ingredient'] == recipe_ingredient].head(1)    
    recipe_ingredients_df.merge(matches_df, how='outer', left_on='ingredient_name', right_on='ingredient')

recipe_ingredients_df

In [ ]:
import requests
import pandas as pd
import numpy as np
import json
import sqlalchemy
from sqlalchemy import create_engine
from flask import Flask, request, render_template, jsonify
import pymysql
pymysql.install_as_MySQLdb()
from config import remote_db_endpoint, remote_db_port, remote_db_name, remote_db_user, remote_db_pwd
from config import x_rapidapi_key, x_rapidapi_host, spoonacular_API

import pprint
import urllib.request



recipe_ids_list = [1554861,1560677,1571559]

capture_list = []
for recipe_id in recipe_ids_list:
    url2 = f"https://api.spoonacular.com/recipes/{recipe_id}/information?apiKey={spoonacular_API}"    
    response = requests.get(url2)
    response_json = response.json()
    capture_list.append(response_json)

capture_list

print(json.dumps(capture_list, indent=4, sort_keys=True))

recipe_ingredients = []
recipe_steps = []

# ingredients stuff
for result in capture_list:                         ################ NOT SURE BUT PROBABLY
    try:
        recipe_id = result['id']
        recipe_title = result['title']        
        analyzedInstructions = result['analyzedInstructions']

    except Exception as e:
        print('--- error with something ---')
        print(e)
        continue 

    instruction_steps = analyzedInstructions[0]['steps']        # Brooke addition

    counter = 0                                                 # Brooke addition

    # INSTRUCTIONS ##############################
    for item in instruction_steps:                              # Brooke addition
        counter = counter + 1                                   # Brooke addition
        step = item['step']                                     # Brooke addition
        numbered_step = f'{counter}. {step}'                    # Brooke addition
        recipe_steps.append(numbered_step)                      # Brooke addition

    # INGREDIENTS ###############################
    for instruction in analyzedInstructions:

        steps = instruction['steps']

        for step in steps:

            ingredients = step['ingredients']

            for ingredient in ingredients:

                ingredient_name = ingredient['name']

                recipe_ingredient = {
                    'recipe_id': recipe_id,
                    'recipe_title': recipe_title,
                    'ingredient_name': ingredient_name
                }

                recipe_ingredients.append(recipe_ingredient)

ingredients_df = pd.DataFrame(recipe_ingredients)

# dedupe ingredients df
# ingredients_df.drop_duplicates()
ingredients_df.drop_duplicates(subset=['ingredient_name'], inplace=True)

ingredients_df

######################## KEEP FOR POSSIBLE USE WITH FUNCTION
# return ingredients_df

cloud_engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")

cloud_conn = cloud_engine.connect()

#%% Querying the database
query = '''
        SELECT DISTINCT
            ingredient,
            price,
            title,
            size
        FROM
            products_subset
        '''

products_subset = pd.read_sql(query, cloud_conn)

products_subset

len(products_subset)

# Renamed to GROCERY DF for clarity
# Cut down to a single return for each ingredient
grocery_df = products_subset
grocery_df.drop_duplicates(subset='ingredient', keep='first', inplace=True)
grocery_df = grocery_df.rename(columns={"title": "ingredient_title"})
print(len(grocery_df))
grocery_df.head()

recipe_ingredients_df = ingredients_df
recipe_ingredients_df = recipe_ingredients_df.rename(columns={"ingredient_name": "ingredient"})
recipe_ingredients_df.head()

new = pd.merge(recipe_ingredients_df, grocery_df, how="inner", on=["ingredient", "ingredient"])
                                                                    
new.head()  

